In [5]:

# imports
from openai import OpenAI
import os
import json

In [ ]:
client = OpenAI()
batch_input_file_path = "path/to/file.jsonl"

if not os.path.exists(batch_input_file_path):
    raise FileNotFoundError(f"Batch input file is not found at : {batch_input_file_path}")

In [ ]:
# create batch input file object
batch_input_file = client.files.create(
    file=open(batch_input_file_path, "rb"), purpose="batch"
)

# print batch file ID.
print(batch_input_file.id)

In [ ]:
# create the job

batch = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={"description": "bulk job"},
)

print(batch.id)
with open("batch_response.json", "w") as fp:
    json.dump(batch, fp, indent=2)

In [ ]:
import time

while True:
    batch_job = client.batches.retrieve(batch.id)
    print(f"Status: {batch_job.status}")
    if batch_job.status == "completed":
        break
    elif batch_job.status == "failed":
        print("Batch failed")
        break
    time.sleep(10)


In [ ]:
output_file_id = batch_job.output_file_id
output_file = client.files.retrieve(output_file_id)
content = client.files.download(output_file_id)
with open("batch_output.jsonl", "wb") as f:
    f.write(content)